# Notebook 02: Limpieza de Datos

**Hito 1 - Análisis de Redes y Consumo de Servicios**

Este notebook realiza:
1. Carga del dataset snapshot desde `data/processed/`
2. Validación de esquema y dominios
3. Normalización de texto (espacios, mayúsculas, acentos)
4. Deduplicación y manejo de nulos
5. Exportación de datos limpios y log de limpieza

## 1. Importación de Librerías

In [42]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Añadir src/ al path
sys.path.insert(0, str(Path.cwd().parent / 'src'))

# Limpiar módulos cargados anteriormente si existen
modules_to_reload = ['config_loader', 'logging_setup', 'io_utils', 'validate', 'cleaning', 'network_prep', 'eda_basic']
for mod in modules_to_reload:
    if mod in sys.modules:
        del sys.modules[mod]

# Importar módulos del proyecto
import config_loader
import logging_setup
import io_utils
import validate
import cleaning

# Imports específicos para usar directamente
from config_loader import load_config, get_absolute_path
from logging_setup import get_etl_logger, log_section
from io_utils import find_data_file, read_data_file, write_data_file
from validate import full_validation
from cleaning import clean_data_pipeline, export_cleaning_log

print("✓ Librerías importadas correctamente")

✓ Librerías importadas correctamente


## 2. Carga de Configuración y Datos

In [43]:
# Cargar configuración
config = load_config()

# Inicializar logger
logger = get_etl_logger(config)
log_section(logger, "NOTEBOOK 02: LIMPIEZA DE DATOS")

print("✓ Configuración cargada")

2025-10-27 23:44:03 | INFO     | etl_pipeline | ================================================================================
2025-10-27 23:44:03 | INFO     | etl_pipeline |   NOTEBOOK 02: LIMPIEZA DE DATOS
2025-10-27 23:44:03 | INFO     | etl_pipeline | ================================================================================
✓ Configuración cargada
2025-10-27 23:44:03 | INFO     | etl_pipeline |   NOTEBOOK 02: LIMPIEZA DE DATOS
2025-10-27 23:44:03 | INFO     | etl_pipeline | ================================================================================
✓ Configuración cargada


In [44]:
# Cargar snapshot desde data/processed/
processed_path = get_absolute_path(config, 'data_processed')
snapshot_file = find_data_file(processed_path, pattern='snapshot')

data = read_data_file(snapshot_file)

logger.info(f"Snapshot cargado: {snapshot_file.name}")
print(f"📁 Datos cargados: {data.shape}")
print(f"\nPrimeras filas:")
display(data.head())

2025-10-27 23:44:03 | INFO     | etl_pipeline | Snapshot cargado: datosRed_snapshot.csv
📁 Datos cargados: (10384, 6)

Primeras filas:
📁 Datos cargados: (10384, 6)

Primeras filas:


,AÑO,PERSONA,TIPO DE SERVICIO,NOMBRE DE LA TAREA,MODALIDAD,COMPLEJIDAD
0,2019,ID_1,SERV_1,TAREA_1.1,PRESENCIAL,BAJA
1,2019,ID_2,SERV_1,TAREA_1.1,PRESENCIAL,BAJA
2,2019,ID_3,SERV_1,TAREA_1.1,PRESENCIAL,BAJA
3,2019,ID_4,SERV_1,TAREA_1.1,PRESENCIAL,BAJA
4,2019,ID_5,SERV_1,TAREA_1.1,PRESENCIAL,BAJA


## 3. Validación Pre-Limpieza

In [45]:
# Validar esquema, dominios y duplicados
key_columns = ['PERSONA', 'TIPO DE SERVICIO', 'AÑO', 'NOMBRE DE LA TAREA', 'MODALIDAD']

validation_report = full_validation(
    data,
    config,
    key_columns=key_columns
)

print("\n" + "="*60)
print("REPORTE DE VALIDACIÓN PRE-LIMPIEZA")
print("="*60)
validation_report.print_report()


REPORTE DE VALIDACIÓN PRE-LIMPIEZA
Estado: ✗ RECHAZADO
Errores: 3
Advertencias: 0

Errores detectados:
  [DOMAIN] Columna MODALIDAD: 10384 registros con valores fuera del dominio
    Detalles: {'valores_invalidos': ['PRESENCIAL', 'VIRTUAL'], 'valores_permitidos': ['Presencial', 'Virtual'], 'registros_afectados': np.int64(10384)}
  [DOMAIN] Columna COMPLEJIDAD: 10384 registros con valores fuera del dominio
    Detalles: {'valores_invalidos': ['MEDIANA', 'BAJA', 'ALTA'], 'valores_permitidos': ['Baja', 'Alta', 'Mediana'], 'registros_afectados': np.int64(10384)}
  [DUPLICATES] 3797 registros duplicados en clave ['PERSONA', 'TIPO DE SERVICIO', 'AÑO', 'NOMBRE DE LA TAREA', 'MODALIDAD']
    Detalles: {'n_duplicados': np.int64(3797)}


## 4. Pipeline de Limpieza

In [46]:
# Ejecutar pipeline completo de limpieza
data_clean, cleaning_log = clean_data_pipeline(
    data,
    config,
    key_columns=key_columns
)

print("\n" + "="*60)
print("LIMPIEZA COMPLETADA")
print("="*60)
print(f"Registros antes:  {cleaning_log.before_stats['n_rows']:,}")
print(f"Registros después: {cleaning_log.after_stats['n_rows']:,}")
print(f"Removidos:        {cleaning_log.before_stats['n_rows'] - cleaning_log.after_stats['n_rows']:,}")


LIMPIEZA COMPLETADA
Registros antes:  10,384
Registros después: 7,955
Removidos:        2,429


## 5. Validación Post-Limpieza

In [47]:
# Revalidar datos limpios
validation_clean = full_validation(
    data_clean,
    config,
    key_columns=key_columns
)

print("\n" + "="*60)
print("REPORTE DE VALIDACIÓN POST-LIMPIEZA")
print("="*60)
validation_clean.print_report()

if validation_clean.passed:
    print("\n✓ Todos los contratos de calidad aprobados")
else:
    print("\n⚠️  Aún existen problemas de calidad")


REPORTE DE VALIDACIÓN POST-LIMPIEZA
Estado: ✓ APROBADO
Errores: 0
Advertencias: 0

✓ Todos los contratos de calidad aprobados


## 6. Exportación de Datos Limpios

In [48]:
# Exportar datos limpios
clean_file_path = processed_path / config['outputs']['datos_limpios']
write_data_file(data_clean, clean_file_path)

logger.info(f"Datos limpios exportados: {clean_file_path}")
print(f"✓ Datos limpios guardados: {clean_file_path.name}")

2025-10-27 23:44:04 | INFO     | etl_pipeline | Datos limpios exportados: /Users/kanyewest/Documents/ComplexNetworks/ComplexNetworks_TP/data/processed/datos_limpios.csv
✓ Datos limpios guardados: datos_limpios.csv
✓ Datos limpios guardados: datos_limpios.csv


## 7. Exportación de Log de Limpieza

In [49]:
# Exportar log de limpieza
reports_path = get_absolute_path(config, 'reports')
export_cleaning_log(cleaning_log, reports_path, config)

print("✓ Logs de limpieza exportados")
print(f"  - {config['outputs']['limpieza_log_csv']}")
print(f"  - {config['outputs']['limpieza_log_md']}")

print("\n" + "="*60)
print("LIMPIEZA FINALIZADA")
print("="*60)

✓ Logs de limpieza exportados
  - limpieza_log.csv
  - limpieza_log.md

LIMPIEZA FINALIZADA
